In [1]:
import numpy as np

# import scipy.special for the sigmoid function ('expit()')
from scipy import special

In [2]:
# neural network class definition
class neuralNetwork:
    
    # initialize the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set the number of nodes in each input, hidden and output layer
        self.i_nodes = inputnodes
        self.h_nodes = hiddennodes
        self.o_nodes = outputnodes
        
        # link weight matrices wih and who
        # weights inside the arrays are w_i_j, where link is from node i to j in the next layer
        # w11 w21
        # w12 w22 etc

        # the simple way:
        # (we substract 0.5 to have values between -0.5 and +0.5)
        # self.wih = (np.random.rand(self.h_nodes, self.i_nodes) - 0.5)
        # self.who = (np.random.rand(self.o_nodes, self.h_nodes) - 0.5)
        
        # fancier way of doing the above is using a normal distribution around zero
        # with a standard deviation related to the number of incoming links to a node:
        # 1/√(number of incoming links) (in other words: the number of incoming links ** -0.5)
        self.wih = np.random.normal(0.0, pow(self.i_nodes, - 0.5), (self.h_nodes, self.i_nodes))
        self.who = np.random.normal(0.0, pow(self.h_nodes, - 0.5), (self.o_nodes, self.h_nodes))
        
        # set the learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function ('expit' in scipy.special package)
        self.activation_function = lambda x: special.expit(x)

        pass
    
    # train the neural network
    def train(inputs_list, targets_list):
        
        # convert inputs_list and targets_list to 2d array (and transpose them)
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals coming into and out of hidden layer
        # inputs 𝗫 = 𝗪 · 𝗜 using the matrix dot product function
        hidden_inputs = np.dot(self.wih, inputs)
        # outputs 𝗢 = sigmoid(𝗫) using sigmoid function as activation function
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals coming into and out of output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # output error is (target output - actual output)
        output_errors = targets - final_outputs
        
        # to train the model to learn from output_errors, we need to adjust
        # the weights and backpropagate adjustments to the previous layers
        # errors coming from the hidden layer is output_errors split by weights,
        # and then recombined again at each hidden node
        hidden_errors = np.dot(self.who.T, output_errors)
        # update the weights for the links between hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        # update the weights for the links between input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass
    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs_list to 2d array and transpose it
        inputs = np.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate signals coming from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate signals coming from the output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    

In [3]:
# number of input, hidden and output nodes
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# learning rate
learning_rate = 0.3

# create instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [4]:
n.query([1.0, 0.5, -1.0])

array([[0.39959274],
       [0.35034366],
       [0.28408972]])